In [782]:

import os
import sys
import pandas as pd
import numpy as np
import create_dicts


code_path = os.getcwd()
data_path = "C:/Users/XHK/Desktop/thesis_code/events_analysis/data/raw/kaggle"

event_type1, event_type2, side, shot_place, shot_outcome, location, \
    bodypart, assist_method, situation = create_dicts.get_dictionaries()

events = pd.read_csv('/'.join([data_path, "events_w_weighted_french.csv"]))
ginf = pd.read_csv('/'.join([data_path, "ginf.csv"]))

events_columns = ['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
                  'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
                  'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
                  'location', 'bodypart', 'assist_method', 'situation', 'fast_break']

ginf_columns = ['id_odsp', 'link_odsp', 'adv_stats', 'date', 'league', 'season',
                'country', 'ht', 'at', 'fthg', 'ftag', 'odd_h', 'odd_d', 'odd_a',
                'odd_over', 'odd_under', 'odd_bts', 'odd_bts_n'] 


Getting dictionaries to analyse events.csv


Some code to determine which league is best to start analysing: 

the variable ginf is a DataFrame created from a .csv which stores general info about each match in the 'events' dataset

We want detailed match-data, which means the match must have adv_stats == True

In [783]:
ginf.groupby(['adv_stats', 'league']).count()


id_odsp  link_odsp  date  season  country    ht    at  fthg  \
adv_stats league                                                                
False     D1           49         49    49      49       49    49    49    49   
          E0          820        820   820     820      820   820   820   820   
          F1           19         19    19      19       19    19    19    19   
          I1           18         18    18      18       18    18    18    18   
          SP1          24         24    24      24       24    24    24    24   
True      D1         1641       1641  1641    1641     1641  1641  1641  1641   
          E0         1300       1300  1300    1300     1300  1300  1300  1300   
          F1         2088       2088  2088    2088     2088  2088  2088  2088   
          I1         2088       2088  2088    2088     2088  2088  2088  2088   
          SP1        2065       2065  2065    2065     2065  2065  2065  2065   

                  ftag  odd_h  odd_d  odd_a  odd_over  odd_under  odd_bts  \
adv_stats league                                                            
False     D1        49     49     49     49         0          0        0   
          E0       820    820    820    820         0          0        0   
          F1        19     19     19     19         0          0        0   
          I1        18     18     18     18         0          0        0   
          SP1       24     24     24     24         0          0        0   
True      D1      1641   1641   1641   1641       153        153      153   
          E0      1300   1300   1300   1300       220        220      220   
          F1      2088   2088   2088   2088       208        208      208   
          I1      2088   2088   2088   2088       207        207      207   
          SP1     2065   2065   2065   2065       189        189      189   

                  odd_bts_n  
adv_stats league             
False     D1              0  
          E0              0  
          F1              0  
          I1              0  
          SP1             0  
True      D1            153  
          E0            220  
          F1            208  
          I1            207  
          SP1           189

France and Italy have the most matches with available events-data in the dataset. 
We pick the French league arbitrarily, moving forward.

first we want to have a column for home-goal-attempts and away-goal-attempts and add it to ginf. 
attempt has an event_type of 1

In [784]:
matches_no_stats_fr = ginf[(ginf.league == 'F1') & (ginf.adv_stats == False)]

matches_fr = ginf[(ginf.adv_stats == True) & (ginf.league == 'F1')]
events_fr = events.loc[events.id_odsp.isin(matches_fr.id_odsp)]



number of attempts by either team. own-goals are ignored.

We add the amount of attempts on goal by the home-team and by the away-team. 
There are around 12 NaN's in those columns. We will deal with them by using the average amount of attempts of the corresponding teams:

In [785]:

#matches_fr['home_attempts'] = np.nan
#matches_fr['away_attempts'] = np.nan

h_att = events_fr[(events_fr.event_type == 1) & (
        events_fr.side == 1) &(event_type2 != 15)].groupby('id_odsp').count()

a_att = events_fr[(events_fr.event_type == 1) & (
    events_fr.side == 2) & (event_type2 != 15)].groupby('id_odsp').count()



In [786]:
matches_fr.index = matches_fr.id_odsp
matches_fr = pd.concat([matches_fr,h_att.event_type],   axis = 1)
matches_fr = matches_fr.rename({'event_type':'home_attempts' }, axis='columns')
matches_fr = pd.concat([matches_fr,a_att.event_type],   axis = 1)
matches_fr = matches_fr.rename({'event_type':'away_attempts'}, axis='columns')

#matches_fr.head()



replacing NaNs  in Home_- and Away_attempts  with the average over all games (this causes minor information-leak from future values to past values, but only around 19 NaNs in the dataset)

In [787]:
home_nans = matches_fr[matches_fr.home_attempts.isna()]['ht']
away_nans = matches_fr[matches_fr.away_attempts.isna()]['at']


# homers = home_nans.values
# for team in homers:
#     matches_fr[matches_fr.ht == team].home_attempts.mean()
#home_nans

for game_id,team in home_nans.iteritems():
    mean_att = matches_fr[matches_fr.ht == team].home_attempts.mean()
    matches_fr.loc[matches_fr.id_odsp == game_id, 'home_attempts'] = mean_att
for game_id, team in away_nans.iteritems():
    mean_att = matches_fr[matches_fr['at'] == team].away_attempts.mean()
    matches_fr.loc[matches_fr.id_odsp == game_id, 'away_attempts'] = mean_att


We proceed by getting the amount of red card events:
In the events.csv, 'location'

First we define a helper function:  

In [788]:
def get_events(events, event_type_nr, type=1 ):
    event_type = 'event_type'
    if type != 1:
        event_type= 'event_type2'
    h_events = events[(events[event_type] == event_type_nr) & (
        events.side == 1)].groupby('id_odsp').count()


    a_events = events[(events[event_type] == event_type_nr) & (
        events.side == 2)].groupby('id_odsp').count()

    return h_events, a_events

def concat_and_rename(df_matches, home_vec, away_vec, name, type = 1):
    event_type = 'event_type'
    if type != 1:
        event_type = 'event_type2'

    home_name = 'home_'+name
    away_name = 'away_'+name 
    df_matches.index = df_matches.id_odsp

    df_matches = pd.concat([df_matches, home_vec[event_type]],   axis=1)
    df_matches = df_matches.rename({event_type: home_name}, axis='columns')
    df_matches = pd.concat([df_matches, away_vec[event_type]],   axis=1)
    df_matches = df_matches.rename({event_type: away_name}, axis='columns')

    # changing nans to zeroes:
    if
    print(f"replacing {sum(df_matches[home_name].isna())} NaNs by {int(df_matches[home_name].fillna(0).mean())} for event =  {name}")
    df_matches.loc[df_matches[home_name].isna(), home_name] = int(
        df_matches[home_name].fillna(0).mean())
    df_matches.loc[df_matches[away_name].isna(), away_name] = int(
        df_matches[away_name].fillna(0).mean())
    return df_matches

home_redcard,away_redcard = get_events(events_fr, 6)
matches_fr = concat_and_rename(matches_fr, home_redcard, away_redcard, name = 'redcard')


replacing 1953 NaNs by 0 for event =  redcard


changing all the NaN's to 0 red_card values: 

Adding the 'conceded penalty' feature:       penalty_conceded has eventnumber 11

In [789]:
home_penalty_conceded, away_penalty_conceded = get_events(events_fr, 11)
matches_fr = concat_and_rename(    matches_fr, home_redcard, away_redcard, name='penalty_conceded')



replacing 1953 NaNs by 0 for event =  penalty_conceded


In [790]:
sum(matches_fr.home_redcard.isna())

0

adding columns for a home_sentoff and away_sentoff features

In [791]:
home_sent, away_sent = get_events(events_fr, 14, type=2)
matches_fr = concat_and_rename(    matches_fr, home_sent, away_sent, name='sentoff')


replacing 1886 NaNs by 0 for event =  sentoff


adding corner_conceded features for both teams.   Corners conceded is lower for away team???

In [792]:
home_corner, away_corner = get_events(events_fr, 2, type = 1)
matches_fr = concat_and_rename(
    matches_fr, home_corner, away_corner, name='corners_taken')


replacing 38 NaNs by 5 for event =  corners_taken


fouls

In [793]:
home_fouls, away_fouls = get_events(events_fr, 3, type=1)
matches_fr = concat_and_rename(    matches_fr, home_fouls, away_fouls, name='fouls')


replacing 14 NaNs by 12 for event =  fouls


In [794]:
home_free_kicks, away_free_kicks = get_events(events_fr, 8 ,type=1 )
matches_fr = concat_and_rename(matches_fr, home_free_kicks, away_free_kicks, name = 'free_kicks')

replacing 14 NaNs by 13 for event =  free_kicks


In [795]:
home_handballs, away_handballs = get_events(events_fr, 10)
matches_fr = concat_and_rename(matches_fr, home_handballs, away_handballs, name = 'handballs')

replacing 1063 NaNs by 0 for event =  handballs


In [796]:
home_offsides, away_offsides = get_events(events_fr, 9)
matches_fr = concat_and_rename(
    matches_fr, home_offsides, away_offsides, name='offsides')


replacing 292 NaNs by 2 for event =  offsides


In [797]:
away_own_goal, home_own_goal  = get_events(events_fr, 15, type = 2)  #away, home switched around because an own_goal commited by home team, is credited to the 'side'-variable of away_team
matches_fr = concat_and_rename(
    matches_fr, away_own_goal, home_own_goal, name='own_goal')


replacing 1993 NaNs by 0 for event =  own_goal


[] Use shot-placement, location and isGoal to determine a weighting for attempts
    [use logistic-regression]
[] Make a weighted-attempts feature

In [798]:
#all the French attempt-events that did not end in own-goals 
french_attempt_events = events[(events.id_odsp.isin(matches_fr.id_odsp)) & (events.event_type ==1) & (events.event_type2 !=15)]
french_attempt_events.groupby('id_odsp side'.split()).sum()

home_weighted_attempts = french_attempt_events[french_attempt_events.side == 1].groupby(
    'id_odsp').sum().french_attempt_weight

away_weighted_attempts = french_attempt_events[french_attempt_events.side == 2].groupby('id_odsp').sum().french_attempt_weight

matches_fr = pd.concat([matches_fr, home_weighted_attempts],   axis=1)
matches_fr = matches_fr.rename(
    {'french_attempt_weight': 'home_weighted_attempts'}, axis='columns')
matches_fr = pd.concat([matches_fr, away_weighted_attempts],   axis=1)
matches_fr = matches_fr.rename(
    {'french_attempt_weight': 'away_weighted_attempts'}, axis='columns')

# changing nans to zeroes:
# matches_fr.loc[matches_fr[home_name].isna(), home_name] = 0
# matches_fr.loc[matches_fr[away_name].isna(), away_name] = 0


Possession-proxy features: 

events that get or need possession to occur: 
attempt, offside, corner, freekick, passing

events that are negatively associated with possession: 
foul, red card, yellow card, andball, penalty

Pass and assists analysis according to DSAA paper:

[1. DONE ] total assist-volume: assist-methods 1,2,3,4              proxy for omega    (passing volume)
[2. ] mean players passing volume (incl. substitutes)               mu_p  
[3. ] variance of players passing volume                            sigma_p 


In [799]:
# TOTAL ASSIST VOLUME PER MATCH

total_assists_home = events_fr[(events_fr['assist_method'] != 0) & (
    events_fr.side == 1)].groupby('id_odsp').count().assist_method

total_assists_away = events_fr[(events_fr['assist_method'] != 0) & (
    events_fr.side == 2)].groupby('id_odsp').count().assist_method

matches_fr = pd.concat([matches_fr, total_assists_home],   axis=1)
matches_fr = matches_fr.rename(
     {'assist_method': 'total_assists_home'}, axis='columns')
matches_fr = pd.concat([matches_fr, total_assists_away],   axis=1)
matches_fr = matches_fr.rename(
     {'assist_method': 'total_assists_away'}, axis='columns')

matches_fr.loc[matches_fr.total_assists_home.isna(), 'total_assists_home'] = 0
#sum(matches_fr.total_assists_away.isna())
matches_fr.loc[matches_fr.total_assists_away.isna(), 'total_assists_away'] = 0
# sum(matches_fr.total_assists_away.isna())


Fixing the NaNs in weighted_attempts by replacing them with the respective teams' mean weighted_attempts

In [800]:
matches_fr.isna().sum()

home_nans = matches_fr[matches_fr.home_weighted_attempts.isna()]['ht']
away_nans = matches_fr[matches_fr.away_weighted_attempts.isna()]['at']


# homers = home_nans.values
# for team in homers:
#     matches_fr[matches_fr.ht == team].home_attempts.mean()
#home_nans

for game_id, team in home_nans.iteritems():
    mean_att = matches_fr[matches_fr.ht == team].home_weighted_attempts.mean()
    matches_fr.loc[matches_fr.id_odsp == game_id, 'home_weighted_attempts'] = mean_att
for game_id, team in away_nans.iteritems():
    mean_att = matches_fr[matches_fr['at'] == team].away_weighted_attempts.mean()
    matches_fr.loc[matches_fr.id_odsp == game_id, 'away_weighted_attempts'] = mean_att


mean_assists 

In [801]:
# home_mean_assists = total_assists_home.copy()/11
# away_mean_assists = total_assists_away.copy()/11

In [802]:

home_assist_stds = pd.DataFrame(events_fr[(events_fr['assist_method'] != 0) & (events_fr['side'] == 1)].groupby(
    'id_odsp event_team player2 '.split()).apply(lambda x: x.assist_method.count()).groupby('id_odsp').std())
home_assist_stds.columns = ['home_assist_stds']
home_assist_stds


,home_assist_stds
id_odsp,
00OX4xFp/,0.500000
00QH2XdM/,0.894427
00nmICd9/,0.816497
00o3l4Ui/,1.691482
02Tk5DSs/,1.414214
...,...
zwHQ9EyO/,0.462910
zwapuIs1/,0.951190
zwg8Qk7J/,0.707107


In [803]:
home_assist_stds = pd.DataFrame(events_fr[(events_fr['assist_method'] != 0) & (events_fr['side'] == 1)].groupby(
    'id_odsp event_team player2 '.split()).apply(lambda x: x.assist_method.count()).groupby('id_odsp').std())
home_assist_stds.columns = ['home_assist_stds']

away_assist_stds = pd.DataFrame(events_fr[(events_fr['assist_method'] != 0) & (events_fr['side'] == 2)].groupby(
    'id_odsp event_team player2 '.split()).apply(lambda x: x.assist_method.count()).groupby('id_odsp').std())
away_assist_stds.columns = ['away_assist_stds']

matches_fr = pd.concat([matches_fr, home_assist_stds],   axis=1)
matches_fr = matches_fr.rename(
    {'assist_method': 'home_assist_stds'}, axis='columns')
matches_fr = pd.concat([matches_fr, away_assist_stds],   axis=1)
matches_fr = matches_fr.rename(
    {'assist_method': 'away_assist_stds'}, axis='columns')

# # changing nans to zeroes:
matches_fr.loc[matches_fr['home_assist_stds'].isna(), 'home_assist_stds'] = 0
matches_fr.loc[matches_fr['away_assist_stds'].isna(), 'away_assist_stds'] = 0
